In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [6]:
STORM_NUMBER = 13
TITLE = "10 March 2022"
SUBTITLE = "8 AM 11 March 2022"
SETPOINT_LOCS = {}
sts = utc(2022, 3, 10, 6)
ets = utc(2022, 3, 11, 20, 0)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [7]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [8]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        geo = gpd.points_from_xy(x=[SETPOINT_LOCS[sp][0]], y=[SETPOINT_LOCS[sp][1]], crs=df.crs)
        df.loc[10000 + sp] = {"geo": geo[0], "val": val, "used_for_analysis": True,
                             "plotme": True}
    df["geo"] = gpd.GeoSeries(df["geo"], crs=df.crs)

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [16]:
def main():
    setpoints = [
        (52, 0),
        (53, 0),
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = []
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=10)
    # draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='220311.png')
    mp.close()

main()

/opt/miniconda3/envs/prod/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/miniconda3/envs/prod/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


      state  wfo   val       lon      lat                              geo  \
0        NM  ABQ  15.0 -105.4400  36.6000  POINT (-486513.546 -917457.183)   
1        KS  GLD  13.0 -101.7500  38.8900  POINT (-151662.197 -677484.635)   
2        CO  GJT  13.0 -107.8300  37.9100  POINT (-687001.368 -755654.058)   
3        KS  GLD  13.0 -101.5900  38.9100  POINT (-137758.040 -675542.329)   
4        KS  GLD  13.0 -100.2400  39.0700   POINT (-20746.719 -659061.413)   
...     ...  ...   ...       ...      ...                              ...   
3862     WI  GRB   0.0  -87.8119  44.3513       POINT (964573.253 481.012)   
3865     WI  GRB   0.0  -87.2292  45.1242     POINT (996751.355 92728.427)   
3866     WI  GRB   0.0  -88.2583  45.9336    POINT (904061.905 169811.769)   
10052   NaN  NaN   0.0       NaN      NaN   POINT (542298.387 -335052.517)   
10053   NaN  NaN   0.0       NaN      NaN   POINT (586808.685 -331346.341)   

       used_for_analysis  nwsli  plotme source  xcell  ycell  
